In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import os
os.environ["MX_DEX_ENV"] = "shadowfork4"
import config

In [ ]:
from context import Context

context = Context()

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)

Start

Control epochs one by one

In [ ]:
from time import sleep
epocs_to_advance = 4
for i in range(epocs_to_advance):
    current_epoch = context.network_provider.proxy.get_network_status(1).current_epoch
    print(f'Current epoch: {current_epoch}')
    context.admin_account.sync_nonce(context.network_provider.proxy)
    sf_control_contract.epochs_fast_forward(context.admin_account, context.network_provider.proxy, 1, 5)
    sleep(60)
    current_epoch = context.network_provider.proxy.get_network_status(1).current_epoch
    print(f'Current epoch: {current_epoch}')

Control epochs in batch

In [ ]:
from time import sleep
epocs_to_advance = 4
current_epoch = context.network_provider.proxy.get_network_status(1).current_epoch
print(f'Current epoch: {current_epoch}')
context.admin_account.sync_nonce(context.network_provider.proxy)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epocs_to_advance, 9)

In [ ]:
from time import sleep

target_epoch = current_epoch + epocs_to_advance
while current_epoch < target_epoch:
    current_epoch = context.network_provider.proxy.get_network_status(1).current_epoch
    print(f'Current epoch: {current_epoch}')
    sleep(10)
print('Reached target epoch!')

Transfer eGLD

In [ ]:
from utils.utils_chain import Account, WrapperAddress, nominated_amount
from utils.utils_tx import endpoint_call

sender = Account(pem_file=config.DEFAULT_ACCOUNTS)
sender.address = WrapperAddress("erd1rf4hv70arudgzus0ymnnsnc4pml0jkywg2xjvzslg0mz4nn2tg7q7k0t6p")
receiver = WrapperAddress("erd1tk3akefn5wmp7tp8vz3tjx9tw3n8mgvefwtqj0fvu2lqyuyl9l0s9z3hrq")
amount = nominated_amount(20)

sender.sync_nonce(context.network_provider.proxy)
endpoint_call(context.network_provider.proxy, 100000, sender, receiver, "cado", [], amount)

In [ ]:
from utils.utils_chain import Account, WrapperAddress, nominated_amount
from utils.utils_tx import endpoint_call

sender = Account(pem_file=config.DEFAULT_ACCOUNTS)
receiver = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
amount = nominated_amount(1)

sender.sync_nonce(context.network_provider.proxy)
endpoint_call(context.network_provider.proxy, 100000, sender, receiver, "cado", [], amount)

Transfer ESDTs

In [ ]:
from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import ESDTToken, multi_esdt_transfer

sender = Account(pem_file=config.DEFAULT_ACCOUNTS)

sender.address = WrapperAddress('')
receiver = WrapperAddress('')

tokens = [
    ESDTToken('TSLA-4C9', 0, 1000000000000000000),
]

sender.sync_nonce(context.network_provider.proxy)
multi_esdt_transfer(context.network_provider.proxy, 1000000, sender, receiver, tokens)